In [23]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import os
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from google.colab import files
from google.colab import drive
import os
from PIL import Image
from keras.layers import Dropout
from keras import regularizers

# /content/drive/MyDrive/pneumonia_dataset/chest_xray/'

# /content/drive/MyDrive/pneumonia_dataset/chest_xray/train_data/
# /content/drive/MyDrive/pneumonia_dataset/chest_xray/test_data/
# /content/drive/MyDrive/pneumonia_dataset/chest_xray/validation_data/

# NORMAL/IM-0007-0001.jpeg
#drive_name = '/content/drive'

In [24]:
class Model:
  def __init__(self,train_dir,test_dir,validation_dir):

    data_gen = ImageDataGenerator(rescale = 1/255)

    self.train_data = data_gen.flow_from_directory(train_dir,
                                              target_size = (150,150),
                                              batch_size = 32,
                                              class_mode = 'binary')

    self.test_data = data_gen.flow_from_directory(test_dir,
                                              target_size = (150,150),
                                              batch_size = 32,
                                              class_mode = 'binary')

    self.validation_data = data_gen.flow_from_directory(validation_dir,
                                                    target_size = (150,150),
                                                    batch_size = 32,
                                                    class_mode = 'binary')
    # initialise Sequential model where you add layers to it
    self.model = keras.Sequential()


    print(f"Training dataset classes: {self.train_data.class_indices}")
    #print(f"Test dataset classes: {self.test_data.class_indices}")
    #print(f"Validation dataset classes: {self.validation_data.class_indices}")


  def init_model(self):
    # first convolutional layer and Maxpool layer.
    self.model.add(keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)))
    self.model.add(keras.layers.MaxPool2D(2,2))

    # second convolutional layer and Maxpool layer.
    self.model.add(keras.layers.Conv2D(64,(3,3),activation = 'relu'))
    self.model.add(keras.layers.MaxPool2D(2,2))

    # third convolutional layer and Maxpool layer.
    self.model.add(keras.layers.Conv2D(128,(3,3),activation = 'relu'))
    self.model.add(keras.layers.MaxPool2D(2,2))

    # #fourth convolutional layer and Maxpool layer.
    # self.model.add(keras.layers.Conv2D(256,(3,3),activation = 'relu'))
    # self.model.add(keras.layers.MaxPool2D(2,2))

    # flattens the image array into a 1D array.
    self.model.add(keras.layers.Flatten())

    # dense layer combines all the features to find important layers.
    self.model.add(keras.layers.Dense(256, activation ='relu',kernel_regularizer= regularizers.l2(0.01)))

    # output later with a single neuron, 0 for Normal and 1 for Pneumonia.
    self.model.add(keras.layers.Dense(1,activation = 'sigmoid'))

    self.model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])


  def predict_image(self,filename):

    image_dir = + filename
    img1 = image.load_img(image_dir,target_size=(150,150))
    plt.imshow(img1)
    Y = image.img_to_array(img1)
    X = np.expand_dims(Y,axis=0)
    val = self.model.predict(X)
    print(val)
    if val == 1:
        plt.xlabel("PNEUMONIA",fontsize=30)
    elif val == 0:

        plt.xlabel("NORMAL",fontsize=30)


  def train_model(self):
    steps= 3706 // 32  # This will give you the full batches
    if 3706 % 32 > 0:
        steps += 1  # Add an extra step if there's a partial batch

    self.model.fit(self.train_data,
            steps_per_epoch = steps,
            epochs = 5,
            validation_data = self.validation_data
            )

  def eval_model(self):
    self.model.evaluate(self.test_data)

  def save_model(self,location,model_name):
    self.model.save('/content/drive/MyDrive/pneumonia_model_tfmodel')
    self.model.save('/content/drive/MyDrive/pneumonia_model_keras.h5')

    #tensorflow model
    self.model.save(f'{location}/{model_name}')
    #keras model save
    self.model.save(f'{location}/{model_name}.h5')

In [25]:
def mount_drive(drive_name):
  drive.mount(drive_name)

In [ ]:
def upload_files():
  uploaded = files.upload()
  for filename in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=filename, length=len(uploaded[filename])))

In [ ]:
def delete_corrupted_images(s_dir):
    bad_images = []
    ext_list = ['.jpg', '.jpeg', '.png', '.bmp']
    for subdir, dirs, files in os.walk(s_dir):
        for file in files:
            ext = os.path.splitext(file)[1]
            if ext.lower() in ext_list:
                try:
                    img_path = os.path.join(subdir, file)
                    img = Image.open(img_path) # open the image file
                    img.verify() # verify that it is, in fact, an image
                except (IOError, SyntaxError) as e:
                    bad_images.append(img_path)
                    os.remove(img_path) # delete the corrupted image
                    print(f"Deleted corrupted image: {img_path}")
    return bad_images

 # Add any other image types you need

train_dir = dir +'/train_data'
test_dir = dir +'/test_data'
val_dir = dir +'/validation_data'

# Delete corrupted images from training, testing and validation sets
train_bad = delete_corrupted_images(train_dir)
test_bad = delete_corrupted_images(test_dir)
val_bad = delete_corrupted_images(val_dir)

print(f"Deleted corrupted Train Images: {len(train_bad)}")
print(f"Deleted corrupted Test Images: {len(test_bad)}")
print(f"Deleted corrupted Validation Images: {len(val_bad)}")


In [29]:
def display_menu():
  print("\n Main Menu: ")
  print("1. Configure Model")
  print("2. Evaluate Model")
  print("3. Save Model")
  print("4. Predict (single)")
  print("5. Delete Corrupted Files")
  print("6. Configure Data")
  print("Q to Quit")
  print("\n")

def data_menu():
  print("\n Data Menu: ")
  print("1. Upload Files")
  print("2. Mount Files")
  print("R to Return to Main Menu")
  print("\n")


In [ ]:
def main():
  print("\n Please enter the details for the Model: ")

  train_directory = input("Enter the directory of the training data")
  test_directory = input("Enter the directory of the testing data")
  validation_directory = input("Enter the directory of the validation data")

  new_model = Model(train_directory,test_directory, validation_directory)

  while True:
    display_menu()
    choice = input("Enter your choice... ")
    if choice == "1":
      new_model.init_model()
      new_model.train_model()

    elif choice == "2":
      new_model.eval_model()

    elif choice == "3":
      new_model.save_model()

    elif choice == "4":
      filename = str(input("Enter the filename you want to predict: "))
      new_model.predict_image(filename)

    elif choice == "5":
      filename = input("Enter the filename you want to start in: ")
      img_corrupted = delete_corrupted_images(filename)
      print(f"Deleted corrupted Images: {len(img_corrupted)}")

    elif choice =="6":
      while True:
        data_menu()
        data_choice = input("Enter your choice... ")
        if data_choice == "1":
          upload_files()
        elif data_choice == "2":
          drive_name = input("Enter Drive Location: ")
          mount_drive(drive_name)
        elif data_choice.lower() == 'r':
          print("Returning...")
          break

    elif choice.lower() == 'q':
      print("****Exiting Program****")
      break

    else:
      print(f"{choice} is not a valid option. Please try again.")




if __name__ == "__main__":
  main()

In [34]:
import argparse
def parse_arguments():
    parser = argparse.ArgumentParser(description='Model Training Script')
    parser.add_argument('--train_dir', type=str, required=True, help='Directory of the training data')
    parser.add_argument('--test_dir', type=str, required=True, help='Directory of the testing data')
    parser.add_argument('--val_dir', type=str, required=True, help='Directory of the validation data')

    return parser.parse_args()

def main():
    args = parse_arguments()

    # Create a new model with directories from command line arguments
    new_model = Model(args.train_dir, args.test_dir, args.val_dir)

    while True:
        display_menu()
        choice = input("Enter your choice... ")
        if choice == "1":
          new_model.init_model()
          new_model.train_model()

        elif choice == "2":
          new_model.eval_model*()

        elif choice == "3":
          new_model.save_model()

        elif choice == "4":
          filename = str(input("Enter the filename you want to predict: "))
          new_model.predict_image(filename)

        elif choice == "5":
          filename = input("Enter the filename you want to start in: ")
          img_corrupted = delete_corrupted_images(filename)
          print(f"Deleted corrupted Images: {len(img_corrupted)}")

        elif choice =="6":
          while True:
            data_menu()
            data_choice = input("Enter your choice... ")
            if data_choice == "1":
              upload_files()
            elif data_choice == "2":
              drive_name = input("Enter Drive Location: ")
              mount_drive(drive_name)
            elif data_choice.lower() == 'r':
              print("Returning...")
              break

        elif choice.lower() == 'q':
          print("****Exiting Program****")
          break

        else:
          print(f"{choice} is not a valid option. Please try again.")


#python script.py --train_dir /path/to/train --test_dir /path/to/test --val_dir /path/to/val
